In [ ]:
using AcausalNets
using QI
using LightGraphs
using LinearAlgebra

In [ ]:
eye(size::Int64) = Diagonal(ones(size))

In [ ]:
roA = Diagonal([1/3, 1/3, 1/3])

roB = Diagonal([1/3, 1/3, 1/3])
roCwAB = Diagonal([
        0,1/2, 1/2, #A=0, B=0
        0,0,1, #A=0, B=1
        #CHANGED to that ro(C | AB) != ro(C | BA) - Originally BE 0,1,0
        0,1,0, #A=0, B=2 
        0,0,1, #A=1, B=0
        1/2,0,1/2, #A=1, B=1
        1,0,0, #A=1, B=2
        0,1,0, #A=2, B=0
        1,0,0, #A=2, B=1
        1/2,1/2,0 #A=2, B=2
        ]); #

var_a = Variable(:a, 3)

var_b = Variable(:b, 3)
var_c = Variable(:c, 3)

sys_a = DiscreteQuantumSystem([var_a], roA)

sys_b = DiscreteQuantumSystem([var_b], roB)

sys_c_ab = DiscreteQuantumSystem([var_a, var_b], [var_c], roCwAB)
classic_an = AcausalNet()

push!(classic_an, sys_a)
push!(classic_an, sys_b)
push!(classic_an, sys_c_ab)
# show(an)

In [ ]:
roAB =1/3*(ket(0,9)+ket(4,9)+ket(8,9))* (bra(0,9)+bra(4,9)+bra(8,9)) 

roC = Diagonal([
        0,1/2,1/2, #A=0, B=0
        0,0,1, #A=0, B=1
        0,1,0, #A=0, B=2
        0,0,1, #A=1, B=0
        1/2,0,1/2, #A=1, B=1
        1,0,0, #A=1, B=2
        0,1,0, #A=2, B=0
        1,0,0, #A=2, B=1
        1/2,1/2,0 #A=2, B=2
        ]); #

var_a = Variable(:a, 3)

var_b = Variable(:b, 3)
var_c = Variable(:c, 3)

sys_ab = DiscreteQuantumSystem([var_a, var_b], roAB)
sys_c_ab = DiscreteQuantumSystem([var_a, var_b], [var_c], roCwAB)
quantum_an = AcausalNet()

push!(quantum_an, sys_ab)
push!(quantum_an, sys_c_ab)

# gplot(Graph(quantum_bn.dag), nodelabel=nodes(quantum_bn))

In [ ]:
roA = Diagonal([.5, .5])
roBwA = Diagonal([.5, .5, .4, .6])
roCwA = Diagonal([.7, .3, .2, .8])
roDwB = Diagonal([.9, .1, .5, .5])
roEwC = Diagonal([.3, .7, .6, .4])
roFwDE = Diagonal([.01, .99, .01, .99, .01, .99, .99, .01])
roGwC = Diagonal([.8, .2, .1, .9])
roHwEG = Diagonal([.05, .95, .95, .05, .95, .05, .95, .05])

var_a = Variable(:a, 2)
var_b = Variable(:b, 2)
var_c = Variable(:c, 2)
var_d = Variable(:d, 2)
var_e = Variable(:e, 2)
var_f = Variable(:f, 2)
var_g = Variable(:g, 2)
var_h = Variable(:h, 2)

sys_a = DiscreteQuantumSystem([var_a], roA)
sys_b = DiscreteQuantumSystem([var_a], [var_b], roBwA)
sys_c = DiscreteQuantumSystem([var_a], [var_c], roCwA)
sys_d = DiscreteQuantumSystem([var_b], [var_d], roDwB)
sys_e = DiscreteQuantumSystem([var_c], [var_e], roEwC)
sys_f = DiscreteQuantumSystem([var_d, var_e], [var_f], roFwDE)
sys_g = DiscreteQuantumSystem([var_c], [var_g], roGwC)
sys_h = DiscreteQuantumSystem([var_e, var_g], [var_h], roHwEG)


example_an = AcausalNet()
push!(example_an, sys_a)
push!(example_an, sys_b)
push!(example_an, sys_c)
push!(example_an, sys_d)
push!(example_an, sys_e)
push!(example_an, sys_f)
push!(example_an, sys_g)
push!(example_an, sys_h)

# gplot(example_bn.dag, nodelabel=nodes(example_bn))
# [v.target for v in example_bn.cpds]


In [ ]:
# roA = Diagonal([.5, .5])

# roBwA = Diagonal([.5, .5, .5, .5])

# roCwB = Diagonal([.5, .5, .5, .5])

# roDwC = Diagonal([.5, .5, .5, .5])
# |
# sys_a = DiscreteQuantumSystem([var_a], roA)
# sys_b = DiscreteQuantumSystem([var_a], [var_b], roBwA)
# sys_c = DiscreteQuantumSystem([var_b], [var_c], roCwB)
# sys_d = DiscreteQuantumSystem([var_c], [var_d], roDwC)

# snake_an = AcausalNet()
# push!(snake_an, sys_a)
# push!(snake_an, sys_b)
# push!(snake_an, sys_c)
# push!(snake_an, sys_d)

# # gplot(snake_an.dag, nodelabel=nodes(snake_bn))

In [ ]:
function moral_graph(an::AcausalNet)
    result = Graph(deepcopy(an.dag))
    for node in an.systems
        for p1 in node.parents
            for p2 in node.parents
                p1_ind = variable_to_node(an, p1)
                p2_ind = variable_to_node(an, p2)
                if(p1_ind!=p2_ind)
                    add_edge!(result, p1_ind, p2_ind)
                end
            end
        end
    end
    return result
end

In [ ]:
an = example_an
moral_an = moral_graph(an)
print(ne(moral_an))
# gplot(moral_bn, nodelabel=nodes(bn))
println(moral_an.fadjlist)
println(variables_names(an))

In [ ]:
is_subset(s1::Set, s2::Set) = intersect(s1, s2) == s1

In [ ]:
function triangulate(mg::Graph, an::AcausalNet)
    mg_copy = [false for _ in vertices(mg)]
    mg = deepcopy(mg)
    nl = systems(an)
    cliques = Vector{Set}()
    while(!all(mg_copy))
        least_edges_to_be_added = Inf
        chosen_vertex = 0
        chosen_cluster = Set()
        for v=1:length(mg_copy)
            if mg_copy[v]
                continue
            else
                cluster = Set()
                for e in edges(mg)
                    if (v==src(e) || v==dst(e)) && !mg_copy[src(e)] && !mg_copy[dst(e)]
                        push!(cluster, src(e))
                        push!(cluster, dst(e))
                    end
                end
                edges_todo = 0
                for v1 in cluster
                    for v2 in cluster
                        if v1 != v2 && !in((v1, v2), edges(mg))
                            edges_todo +=1
                        end
                    end
                end
                edges_todo /= 2
                
                if edges_todo < least_edges_to_be_added || 
                    ((edges_todo == least_edges_to_be_added) && 
                        (prod([ncategories(systems(an)[n]) for n in cluster]) <= prod([ncategories(systems(an)[n]) for n in chosen_cluster])))                      
                    least_edges_to_be_added = edges_todo
                    chosen_vertex = v
                    chosen_cluster = cluster
                end
            end       
        end
        chosen_nodes = Set([nl[n] for n in chosen_cluster])
        if !any([is_subset(chosen_nodes, clique) for clique in cliques])
            push!(cliques, chosen_nodes)
        end
        mg_copy[chosen_vertex] = true
        for v1 in chosen_cluster
            for v2 in chosen_cluster
                if v1 != v2 && !in((v1, v2), edges(mg))
                    add_edge!(mg, v1, v2)
                end
            end
        end     
    end
    cliques = [sort([c for c in clique], by=c -> system_to_node(an, c)) for clique in cliques]
    return mg, cliques
end

In [ ]:
println(ne(moral_an))

tri_moral_an, cliques = triangulate(moral_an,an)
println(ne(tri_moral_an))

# before, cliques were sets
# they are now transformed to vectors and their ordering is the same 
# as in their bayes_net DAG
# println(cliques)
# gplot(tri_moral_an ,nodelabel=nodes(an))

println(tri_moral_an.fadjlist)
println(variables_names(an))

In [ ]:
struct JoinTree
    graph::Graph
    clusters::Vector{Vector{DiscreteQuantumSystem}}
    vertex_to_num::Dict{Int64, AbstractMatrix}
    edge_to_num::Dict{Set{Int}, AbstractMatrix}
    
end

In [ ]:
function sepset_cost(sepset::Set{DiscreteQuantumSystem}, an::AcausalNet)
    weights = [ncategories(s) for s in sepset]
    
    return 0
end

In [ ]:
function sepset_comparator(c1, c2)
    sepset = intersect(c1, c2)
    -length(sepset)
end

In [ ]:
cluster_size(cluster::Vector{DiscreteQuantumSystem}, an::AcausalNet) = prod([ncategories(s) for s in cluster])

In [ ]:
function make_join_tree(clusters::Vector{Vector{DiscreteQuantumSystem}}, an::AcausalNet)
    candidate_sepsets = []
    trees = Dict([c => c for c in clusters])
    chosen_sepsets = Set()
    Dict([v => 1.0 for v=1:length(clusters)])
    
    result = JoinTree(
        Graph(length(clusters)), 
        clusters, 
        Dict([v => eye(cluster_size(clusters[v], an)) for v=1:length(clusters)]),
        Dict()
    )
    for c1 = 1:length(clusters)
        for c2= 1:length(clusters)
            if c1 != c2
                push!(candidate_sepsets,(c1, c2))
            end
        end
    end
    candidate_sepsets = sort(candidate_sepsets, by=c -> sepset_comparator(clusters[c[1]], clusters[c[2]]))
    i = 1
    n = length(clusters)

    while length(chosen_sepsets) < n-1
        i1, i2 = candidate_sepsets[i]
        c1, c2 = clusters[i1], clusters[i2]
        sepset = intersect(c1, c2)
        if (trees[c1] != trees[c2]) && !any([sepset==s for s in chosen_sepsets])
            push!(chosen_sepsets, sepset)
            trees[c1] = trees[c2] = union(c1, c2)
#             println(c1, " ", c2, " ", sepset)
            add_edge!(result.graph, i1, i2)
            push!(result.edge_to_num, Set([i1, i2]) => eye(cluster_size(sepset, an)))
        end
        i += 1
    end
    result
end

In [ ]:
jt = make_join_tree(cliques, an)
jt.clusters

In [ ]:
# gplot(jt.graph, nodelabel=[join([string(v) for v in c]) for c in jt.clusters])

In [ ]:
function parent_systems(an::AcausalNet, ds::DiscreteQuantumSystem)
    result = DiscreteQuantumSystem[]
    for p in parents(ds)
        parent_system = systems(an)[variable_to_node(an, p)]
        if !(parent_system in result)
            push!(result, parent_system)
        end
    end
    return result
end

In [ ]:
function family(an::AcausalNet, dqs::DiscreteQuantumSystem)
    union(Set([dqs]), Set(parent_systems(an, dqs)))
end

In [ ]:
print(jt.clusters[1])
jt.vertex_to_num[1]

In [ ]:
event(system::AbstractMatrix, e::AbstractMatrix) = (e * system * e) / tr(e * system)

In [ ]:
AcausalNets.Systems.variables(cluster::Vector{DiscreteQuantumSystem}) = return [dqs.variables for dqs in cluster] 


In [ ]:
[variables(c) for c in jt.clusters]

In [ ]:
shallowcopy(jt::JoinTree) = JoinTree(
        jt.graph, 
        jt.clusters, 
        deepcopy(jt.vertex_to_num), 
        deepcopy(jt.edge_to_num)
    )

In [ ]:
function initialize(jt::JoinTree, an::AcausalNet, assignments::Dict=Dict())
    jt = shallowcopy(jt)
    for v1 in systems(an)
#         println(v1.variables)
        parent_cluster_ind = [c for c=1:length(jt.clusters) if is_subset(family(an, v1), Set(jt.clusters[c]))][1]
        mul_elem = eye(1)
        multiplied_indices = [] 
        multiplied_dimensions = []
#         println(variables(v1), " chooses ", variables(jt.clusters[parent_cluster_ind]))
        
        for v2 in [v for v in systems(an) if v in jt.clusters[parent_cluster_ind]]
            factor = eye(1)
            if v1 == v2
                assignment_factor = eye(1)
                for p in parent_systems(an, v2)
                    p_index = system_to_node(an, p)
                    push!(multiplied_indices, p_index)
                    push!(multiplied_dimensions, ncategories(p))
                    assignment_factor = kron(assignment_factor, eye(ncategories(p)))
                end
                for v in variables(v2)
                    assignment = get(assignments, v, eye(ncategories(v)))
                    assignment_factor = kron(assignment_factor, assignment)
                end
                factor = event(v2.distribution, assignment_factor) * tr(v2.distribution)
                push!(multiplied_indices, system_to_node(an, v2))
                push!(multiplied_dimensions, ncategories(v2))
            elseif !in(v2, parent_systems(an, v1))
                factor = eye(ncategories(v2))
                push!(multiplied_indices, system_to_node(an, v2))
                push!(multiplied_dimensions, ncategories(v2))
            end
            mul_elem = kron(mul_elem, factor)
        end
#         println(variables(v1), " has chosen")
        sorted_multiplied_indices = sort(multiplied_indices)
        right_order = [findfirst([m == mul for mul in multiplied_indices]) for m in sorted_multiplied_indices]
        
#         println([as.cpds[i].target for i in multiplied_indices])
#         println(multiplied_indices)
#         println([as.cpds[i].target for i in sorted_multiplied_indices])
#         println(sorted_multiplied_indices)
#         println(right_order)
#         println("-----")
        multiplied_dimensions = [multiplied_dimensions[r] for r in right_order]
#         mul_elem = permute_systems(mul_elem, multiplied_dimensions, right_order)

        previous_init = jt.vertex_to_num[parent_cluster_ind]
        jt.vertex_to_num[parent_cluster_ind] = star(mul_elem, previous_init)
    end
#     println()
    jt
end

In [ ]:
init_jt = initialize(jt, an);
# print(init_jt.cliques)
# real(init_jt.vertex_to_num[5])

In [ ]:
roCwA

In [ ]:
# from this it would seem that currently node [a, d, c] 
# holds conditional probability ADCwD

a_factor = kron(kron(roA, eye(ncategories(sys_d))), eye(ncategories(sys_c)))
c_factor = kron(eye(ncategories(sys_d)), roCwA)
c_factor = permute_systems(c_factor, [2,2,2], [2,1,3])
ADCwD = star(c_factor, a_factor)
tr(ADCwD)
ptrace(ADCwD, [2,2,2], [1,2])

In [ ]:
roBwA

In [ ]:
# [a, b, d] holds ABDwA
b_factor = kron(roBwA, eye(ncategories(sys_d)))
d_factor = kron(eye(ncategories(sys_a)), roDwB)

ABDwA = star(b_factor, d_factor)
# trace(ABDwA)
ptrace(ABDwA, [2,2,2], [3])

In [ ]:
init_jt.clusters

This is a raw message pass

In [ ]:
ABDwA = init_jt.vertex_to_num[6] #ABD
# ptrace(BDwA, [2,2,2], [2,3])

In [ ]:
ADCwD = init_jt.vertex_to_num[5] #ADC
# ptrace(ACwD, [2,2,2], [2,3])
# trace(ADCwD)


In [ ]:
# message from ADC to ABD
ADwD = ptrace(ADCwD, [2,2,2], [3])

In [ ]:
# fitting the message into ABD
ADBwBD = kron(ADwD, eye(ncategories(sys_b)))
ABDwBD = permute_systems(ADBwBD, [2,2,2], [1, 3, 2])

In [ ]:
ABD = star(ABDwA, ABDwBD)
# ptrace(ABD, [2,2,2], [2,3])
tr(ABD)
# ptrace(ABD, [2,2,2], [])

In [ ]:
# message from ABD to ADC
AD = ptrace(ABD, [2,2,2], [2])
msg = unstar(AD,ADwD)

In [ ]:
# fitting the message into ADC
ADCwC = kron(msg, eye(ncategories(sys_c)))
tr(ADCwC)

In [ ]:
ADC = star(ADCwD, ADCwC)
tr(ADC)


In [ ]:
ptrace(ADC, [2,2,2], [1,3])

In [ ]:
function single_message_pass(from_ind::Int, to_ind::Int, jt::JoinTree, an::AcausalNet)
    jt = shallowcopy(jt)
    if (from_ind, to_ind) in edges(jt.graph)
        println("message from ", variables(jt.clusters[from_ind]), " to ", variables(jt.clusters[to_ind]))
        cluster_from = jt.clusters[from_ind]
        cluster_to = jt.clusters[to_ind]
        sepset = intersect(cluster_from, cluster_to)
#         println(cluster_from, " ", sepset, " ", cluster_to)
        to_trace_out_sym = setdiff(cluster_from, sepset)
        to_trace_out_ind = [findfirst([s==sym for sym in cluster_from]) for s in to_trace_out_sym]
#         println("trace_out ", to_trace_out_ind)
        from_variables_sizes = [ncategories(v) for v in cluster_from]
#         println(from_variables_sizes)
        cluster_from_num = jt.vertex_to_num[from_ind]
        old_sepset_num = jt.edge_to_num[Set([from_ind, to_ind])]
        new_sepset_num = ptrace(cluster_from_num, from_variables_sizes, to_trace_out_ind)
#         println(new_sepset_num)
#         println(old_sepset_num)
        
#         println(jt.edge_to_num[Set([from_ind, to_ind])])
        jt.edge_to_num[Set([from_ind, to_ind])] = new_sepset_num 
#         println(jt.edge_to_num[Set([from_ind, to_ind])])
        
        cluster_to_num = jt.vertex_to_num[to_ind]
        
#         message = event(old_sepset_num, new_sepset_num) 
        message = unstar(new_sepset_num, old_sepset_num)
        
        message_sym = Vector(sepset)
#         println(message_sym)
        for v in cluster_to
            if !in(v, message_sym)
                push!(message_sym, v)
                message = kron(message, eye(ncategories(v)))
            end
        end
#         println(message_sym)
        message_sorted_inds = [findfirst([s==sym for sym in message_sym]) for s in systems(an) if s in message_sym]
#         println(message_sorted_inds) 
        message_dims = [ncategories(s) for s in message_sym]
        message_sorted = permute_systems(message, message_dims, message_sorted_inds )
        jt.vertex_to_num[to_ind] = star(cluster_to_num, message_sorted) #* trace(cluster_to_num)
#         jt.vertex_to_num[to_ind] = star(cluster_to_num, message_sorted) 
    end
    return jt
end

In [ ]:
[variables(c) for c in jt.clusters]

In [ ]:
init_jt.edge_to_num
init_jt.vertex_to_num[5]
#bcd
ptrace(init_jt.vertex_to_num[5], [2,2,2], [2,3])

In [ ]:
init_jt.vertex_to_num[6]
#abc
ptrace(init_jt.vertex_to_num[6], [2,2,2], [2,3])

In [ ]:
# bcd -> abc
passed = single_message_pass(5, 6, init_jt, an);

In [ ]:
passed.vertex_to_num[5]

In [ ]:
passed.vertex_to_num[6]

In [ ]:
ptrace(passed.vertex_to_num[5], [2,2,2], [2, 3])

In [ ]:
# abc -> bcd 
passed_2 = single_message_pass(6, 5, passed, an);
passed_2.vertex_to_num[5]
ptrace(passed_2.vertex_to_num[5], [2,2,2], [1, 3])

In [ ]:
function collect_evidence(cluster_ind::Int, cluster_marks::Vector{Bool}, jt::JoinTree, an::AcausalNet)
    cluster_marks[cluster_ind] = false
    for neighbor in neighbors(jt.graph, cluster_ind)
        if cluster_marks[neighbor]
#             println(jt.clusters[cluster_ind], " ", jt.clusters[neighbor])
            jt, cluster_marks = collect_evidence(neighbor, cluster_marks, jt, an)
            jt = single_message_pass(neighbor, cluster_ind, jt, an)
            # pass a message from neighbor to cluster_ind
        end
        
    end
    jt = shallowcopy(jt)
    jt, cluster_marks
    
end

In [ ]:
function distribute_evidence(cluster_ind::Int, cluster_marks::Vector{Bool}, jt::JoinTree, an::AcausalNet)
    cluster_marks[cluster_ind] = false
    for neighbor in neighbors(jt.graph, cluster_ind)
        if cluster_marks[neighbor]
            jt = single_message_pass(cluster_ind, neighbor, jt, an)
            # pass a message from cluster_ind to neighbor
        end
    end
    for neighbor in neighbors(jt.graph, cluster_ind)
        if cluster_marks[neighbor]
            jt, cluster_mars = distribute_evidence(neighbor, cluster_marks, jt, an)
        end
    end
    jt = shallowcopy(jt)
    jt, cluster_marks
end

In [ ]:
function global_propagation(jt::JoinTree, an::AcausalNet)
    jt = shallowcopy(jt)
    cluster_marks = [true for c in jt.clusters]
    arbitrary_cluster_ind = 1
#     println("Collect evidence")
    jt, cluster_marks = collect_evidence(arbitrary_cluster_ind, cluster_marks, jt, an)
    cluster_marks = [true for c in jt.clusters]
#     println("Distribute evidence")
    jt, cluster_marks = distribute_evidence(arbitrary_cluster_ind, cluster_marks, jt, an)
    return jt
end

In [ ]:
propagated_jt = global_propagation(init_jt, an)
# propagated_jt = global_propagation(propagated_jt, bn)

In [ ]:
[variables(c) for c in propagated_jt.clusters]

In [ ]:
BCD = propagated_jt.vertex_to_num[5]

In [ ]:
ptrace(BCD, [2,2,2], [1,3])

In [ ]:
ABC = propagated_jt.vertex_to_num[6]

In [ ]:
ptrace(ABD, [2,2,2], [1,2])

In [ ]:
# CDE
ptrace(propagated_jt.vertex_to_num[4], [2,2,2], [2,3])


# To sum everything up

In [ ]:
an = example_an
# gplot(bn.dag, nodelabel=nodes(bn))

In [ ]:
moral_g = moral_graph(an)
tri_moral_graph, cliques = triangulate(moral_g, an)
jt = make_join_tree(cliques, an);

In [ ]:
a = 0
a_oper = ketbra(a,a,2)
assignments = Dict([
        var_a => a_oper
        ])

In [ ]:
initialized_jt = initialize(jt, an, assignments)
propagated_jt = global_propagation(initialized_jt, an)
propagated_jt.clusters

In [ ]:
aBC = propagated_jt.vertex_to_num[6]
ptrace(aBC, [2,2,2], [2,3])